In [2]:
## Next steps:
## Add more data -- Looks like I might need to use beautiful soup to get a full cbb schedule
## Make better model
## Gridsearch

In [3]:
from sklearn import tree #decision tree
from sklearn.ensemble import RandomForestRegressor #random Forest
from sklearn.model_selection import cross_val_score # evaluate our model
from sklearn.model_selection import GridSearchCV # gridsearch to find best hyper parameters
import pandas as pd
import numpy as np

In [4]:
### Data Exploration
np.random.seed(1)
df = pd.read_csv('2020.csv', header=[0,1])
df.head(2)

Unnamed: 0_level_0 Unnamed: 1_level_0 Overall                             \
                  Rk             School       G   W   L   W-L%   SRS   SOS   
0                  1  Abilene Christian      23  13  10  0.565 -2.90 -5.05   
1                  2          Air Force      25   9  16  0.360 -0.53  2.88   

  Conf.      ... School Totals                                                
      W   L  ...            FT  FTA    FT%  ORB  TRB  AST  STL BLK  TOV   PF  
0     8   4  ...           366  496  0.738  262  791  344  212  60  328  485  
1     3  10  ...           387  519  0.746  178  810  354  124  39  305  424  

[2 rows x 34 columns]

In [5]:
### Data Cleaning

### Only want a single header
df = pd.read_csv('2020.csv', header=[1])
df.head(2)

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,23,13,10,0.565,-2.90,-5.05,8,4,...,366,496,0.738,262,791,344,212,60,328,485
1,2,Air Force,25,9,16,0.360,-0.53,2.88,3,10,...,387,519,0.746,178,810,354,124,39,305,424


In [6]:
#rename w.1 etc
df.rename(columns={"W.1": "Conf W", "W.2": "Home W", "W.3": "Away W"}, inplace=True)
df.rename(columns={"L.1": "Conf L", "L.2": "Home L", "L.3": "Away L"}, inplace=True)
df.head(1)

,Rk,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,23,13,10,0.565,-2.9,-5.05,8,4,...,366,496,0.738,262,791,344,212,60,328,485


In [7]:
df.drop(['Unnamed: 16', 'Rk'], axis=1, inplace = True)
df.head(4)

,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,Home W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,23,13,10,0.565,-2.90,-5.05,8,4,10,...,366,496,0.738,262,791,344,212,60,328,485
1,Air Force,25,9,16,0.360,-0.53,2.88,3,10,6,...,387,519,0.746,178,810,354,124,39,305,424
2,Akron,24,18,6,0.750,7.97,-0.53,8,3,12,...,358,461,0.777,236,896,329,117,68,320,421
3,Alabama A&M,22,6,16,0.273,-18.22,-8.27,3,8,4,...,271,388,0.698,217,757,238,132,44,258,387


In [18]:
### Data Exploration of new table
schedule_df_full = pd.read_csv('2019_2020_Schedule.csv')
schedule_df_full.drop('Unnamed: 0',axis=1, inplace=True)
schedule_df_full.tail()

,Date,visitor_neutral,PTS,home_neutral,PTS.1
5410,2020-03-02,Idaho State,78,Weber State,70
5411,2020-03-02,Maryland-Eastern Shore,60,Howard,62
5412,2020-03-02,Mississippi Valley State,61,Grambling,81
5413,2020-03-02,Norfolk State,79,Delaware State,73
5414,2020-03-02,North Carolina A&T,76,South Carolina State,65


In [20]:
### Drop unneeded column Date
schedule_df_full.drop(['Date'], axis=1, inplace=True)

In [21]:
schedule_df_full.tail()

,visitor_neutral,PTS,home_neutral,PTS.1
5410,Idaho State,78,Weber State,70
5411,Maryland-Eastern Shore,60,Howard,62
5412,Mississippi Valley State,61,Grambling,81
5413,Norfolk State,79,Delaware State,73
5414,North Carolina A&T,76,South Carolina State,65


In [24]:

combine = pd.merge(schedule_df_full,df, left_on='visitor_neutral', right_on='School')
combine.head()

,visitor_neutral,PTS,home_neutral,PTS.1,School,G,W,L,W-L%,SRS,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Michigan State,62,Kentucky,69,Michigan State,25,17,8,0.68,21.44,...,354,470,0.753,281,1027,445,132,132,317,417
1,Michigan State,76,Seton Hall,73,Michigan State,25,17,8,0.68,21.44,...,354,470,0.753,281,1027,445,132,132,317,417
2,Michigan State,75,UCLA,62,Michigan State,25,17,8,0.68,21.44,...,354,470,0.753,281,1027,445,132,132,317,417
3,Michigan State,72,Oakland,49,Michigan State,25,17,8,0.68,21.44,...,354,470,0.753,281,1027,445,132,132,317,417
4,Michigan State,77,Northwestern,72,Michigan State,25,17,8,0.68,21.44,...,354,470,0.753,281,1027,445,132,132,317,417


In [28]:
Schedule_and_stats = pd.merge(combine, df, left_on = 'home_neutral', right_on = 'School')
Schedule_and_stats.head()

,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,Michigan State,62,Kentucky,69,Michigan State,25,17,8,0.680,21.44,...,456,578,0.789,233,910,335,124,125,304,443
1,Louisville,70,Kentucky,78,Louisville,24,21,3,0.875,19.03,...,456,578,0.789,233,910,335,124,125,304,443
2,Georgia Tech,53,Kentucky,67,Georgia Tech,24,11,13,0.458,9.17,...,456,578,0.789,233,910,335,124,125,304,443
3,Utah,69,Kentucky,66,Utah,23,14,9,0.609,9.47,...,456,578,0.789,233,910,335,124,125,304,443
4,Mount St. Mary's,62,Kentucky,82,Mount St. Mary's,24,10,14,0.417,-8.07,...,456,578,0.789,233,910,335,124,125,304,443


In [29]:
X = Schedule_and_stats.drop(["visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1)
Y = Schedule_and_stats[["PTS","PTS.1"]]

In [30]:
reg = tree.DecisionTreeRegressor(random_state=0)
reg = reg.fit(X, Y)

In [44]:
### cross validations
print(cross_val_score(reg, X, Y, cv=5))

#First model performs HORRIBLY!

//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "mul

[-0.60536357 -0.74695204 -0.57380704 -0.57869656 -0.55379577]


//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [45]:
# use grid search to determine decision tree depth
params = {"max_depth":range(1,10)}
treereg = tree.DecisionTreeRegressor(random_state=0)
reg2 = GridSearchCV(estimator = treereg, param_grid=params)
reg2.fit(X,Y)
reg2.best_params_


//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent wit

//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "mul

{'max_depth': 4}

In [46]:
means = reg2.cv_results_['mean_test_score']
stds = reg2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()


0.035 (+/-0.011) for {'max_depth': 1}

0.068 (+/-0.030) for {'max_depth': 2}

0.111 (+/-0.013) for {'max_depth': 3}

0.122 (+/-0.059) for {'max_depth': 4}

0.086 (+/-0.071) for {'max_depth': 5}

0.044 (+/-0.075) for {'max_depth': 6}

-0.001 (+/-0.049) for {'max_depth': 7}

-0.064 (+/-0.077) for {'max_depth': 8}

-0.171 (+/-0.085) for {'max_depth': 9}



In [47]:
reg3 = RandomForestRegressor(random_state=0)
reg3.fit(X,Y)
print(cross_val_score(reg3, X, Y, cv=5))

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. 

[0.13223583 0.13835643 0.17977283 0.17721487 0.18591984]


//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [48]:
Schedule_and_stats.drop(["visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1).head()

,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,25,17,8,0.680,21.44,9.72,9,5,10,2,...,456,578,0.789,233,910,335,124,125,304,443
1,24,21,3,0.875,19.03,6.37,12,1,14,1,...,456,578,0.789,233,910,335,124,125,304,443
2,24,11,13,0.458,9.17,9.21,5,8,6,6,...,456,578,0.789,233,910,335,124,125,304,443
3,23,14,9,0.609,9.47,6.51,5,6,10,1,...,456,578,0.789,233,910,335,124,125,304,443
4,24,10,14,0.417,-8.07,-6.46,6,5,7,3,...,456,578,0.789,233,910,335,124,125,304,443


In [50]:
#get game to predict.  Tomorrow's game michigan vs maryland 
test = pd.read_csv("test.csv")
test.drop("Unnamed: 0", axis=1, inplace = True)
test.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,Michigan,NaN,Maryland,NaN,Michigan,23,14,9,0.609,17.15,...,392,530,0.74,276,938,314,114,112,292,355


In [51]:
### BOO!  My model predicts a maryland win 71.6 to 58.2
reg3.predict(test.drop(columns=["Visitor/Neutral","Home/Neutral","PTS","PTS.1","School_x","School_y"]))

array([[58.2, 71.6]])